In [230]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [119]:
start_time = int(datetime(2021,4,20).timestamp())
end_time = int(datetime(2021,8,17).timestamp())

url = f'https://eodhd.com/api/intraday/{ticker}'
params = {
    'api_token': '690ac07599b843.88149106',
    'fmt': 'json',
    'interval': '1m',
    'from': start_time,
    'to': end_time

}
data = requests.get(url, params=params).json() 
pd.json_normalize(data)

,timestamp,gmtoffset,datetime,open,high,low,close,volume
0,1618905600,0,2021-04-20 08:00:00,415.66,415.72,415.59,415.59,5975
1,1618905720,0,2021-04-20 08:02:00,415.51,415.52,415.51,415.52,1846
2,1618905900,0,2021-04-20 08:05:00,415.41,415.41,415.39,415.40,526
3,1618905960,0,2021-04-20 08:06:00,415.50,415.50,415.50,415.50,500
4,1618906080,0,2021-04-20 08:08:00,415.49,415.49,415.49,415.49,500
...,...,...,...,...,...,...,...,...
64086,1629158100,0,2021-08-16 23:55:00,446.79,446.79,446.79,446.79,421
64087,1629158160,0,2021-08-16 23:56:00,446.81,446.81,446.81,446.81,615
64088,1629158220,0,2021-08-16 23:57:00,446.79,446.79,446.74,446.74,3862
64089,1629158280,0,2021-08-16 23:58:00,446.75,446.75,446.74,446.74,2463


In [ ]:
all_df = []
all_time_period = [(datetime(2019, 1, 1), datetime(2019,12,31)), 
                   (datetime(2020, 1, 1), datetime(2020,12,31))]#, (), (), (), ()]

ticker = "SPY.US"
current_datetime = datetime(2019, 1, 1)
end_datetime = datetime(2025, 11, 12)


while int(current_datetime.timestamp()) <= int(end_datetime.timestamp()):

    #end_time = int(datetime(2025, 11, 12).timestamp())
    #start_time = int(datetime(2015, 1, 1).timestamp())

    #end_time = int((datetime(2025, 11, 12) - timedelta(days=i)).timestamp())
    #start_time = int((datetime(2025, 11, 12) - timedelta(days=i+600)).timestamp())

    start_time = int(current_datetime.timestamp())
    end_time = int((current_datetime + timedelta(days=120)).timestamp())

    url = f'https://eodhd.com/api/intraday/{ticker}'
    params = {
        'api_token': '690ac07599b843.88149106',
        'fmt': 'json',
        'interval': '1m',
        'from': start_time,
        'to': end_time

    }
    data = requests.get(url, params=params).json()  
    all_df.append(pd.json_normalize(data))

    current_datetime += timedelta(days=120, minutes=1)


In [275]:
final_df = pd.concat(all_df, ignore_index=True)

In [276]:
final_df = final_df.drop_duplicates(subset=['datetime'])

In [599]:
final_df

In [278]:
final_df['datetime'] = pd.to_datetime(final_df['datetime'])
final_df['diff'] = final_df['datetime'].diff().dt.total_seconds() / 60

In [254]:
final_df[(final_df['diff'] < 200) & (final_df['diff'] > 1)]

,timestamp,gmtoffset,datetime,open,high,low,close,volume,diff
18,1546420740,0,2019-01-02 09:19:00,245.64,245.64,245.64,245.64,169,2.0
20,1546420920,0,2019-01-02 09:22:00,245.66,245.68,245.64,245.64,2300,2.0
26,1546421340,0,2019-01-02 09:29:00,245.93,245.99,245.93,245.99,1209,2.0
29,1546421580,0,2019-01-02 09:33:00,245.97,246.03,245.97,246.03,469,2.0
32,1546421820,0,2019-01-02 09:37:00,246.04,246.04,246.00,246.00,969,2.0
...,...,...,...,...,...,...,...,...,...
1393865,1758617760,0,2025-09-23 08:56:00,666.88,666.89,666.85,666.89,153,2.0
1393987,1758625140,0,2025-09-23 10:59:00,666.61,666.61,666.59,666.59,1718,2.0
1395823,1758792840,0,2025-09-25 09:34:00,660.91,660.99,660.90,660.99,545,2.0
1413113,1761038820,0,2025-10-21 09:27:00,670.84,670.91,670.81,670.89,621,2.0


In [279]:
final_df['datetime'].dt.date

0          2019-01-02
1          2019-01-02
2          2019-01-02
3          2019-01-02
4          2019-01-02
              ...    
1431279    2025-11-15
1431280    2025-11-15
1431281    2025-11-15
1431282    2025-11-15
1431283    2025-11-15
Name: datetime, Length: 1431284, dtype: object

In [314]:
group['datetime'].min()

Timestamp('2025-11-15 00:00:00')

In [496]:
filled = []
for date, group in final_df.groupby(final_df['datetime'].dt.date):
    #market_open_time = group['datetime'].min()
    market_close_time = group['datetime'].max()
    intraday_range = pd.date_range(
        start=f"{date} 09:30:00",
        end=f"{date} {market_close_time.time()}",
        freq='1T'
    )
    intraday_df = group.reindex(intraday_range)
    filled.append(intraday_df)

df_filled = pd.concat(filled)
df_filled = df_filled.drop('datetime', axis=1).reset_index().rename(columns={'index': 'datetime'})[['datetime']]

/var/folders/s_/58nvglhd2pl1yr8404nm038w0000gn/T/ipykernel_10442/489615699.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  intraday_range = pd.date_range(


In [497]:
final_df['diff'].value_counts()

diff
1.0       1329008
2.0         59035
3.0         20256
4.0          9029
5.0          4745
           ...   
3487.0          1
3365.0          1
3399.0          1
1920.0          1
4980.0          1
Name: count, Length: 94, dtype: int64

In [498]:
new_df = pd.merge(df_filled, final_df, how='left', on='datetime')

In [500]:
new_df

,datetime,timestamp,gmtoffset,open,high,low,close,volume,diff
0,2019-01-02 09:30:00,1.546421e+09,0.0,245.910,245.91,245.8800,245.880,979.0,1.0
1,2019-01-02 09:31:00,1.546421e+09,0.0,245.870,245.87,245.8700,245.870,173.0,1.0
2,2019-01-02 09:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-02 09:33:00,1.546422e+09,0.0,245.970,246.03,245.9700,246.030,469.0,2.0
4,2019-01-02 09:34:00,1.546422e+09,0.0,246.050,246.05,246.0500,246.050,300.0,1.0
...,...,...,...,...,...,...,...,...,...
1480519,2025-11-14 23:55:00,1.763164e+09,0.0,672.890,672.89,672.8400,672.840,1729.0,1.0
1480520,2025-11-14 23:56:00,1.763165e+09,0.0,672.840,672.89,672.8300,672.860,374.0,1.0
1480521,2025-11-14 23:57:00,1.763165e+09,0.0,672.840,672.88,672.8400,672.865,203.0,1.0
1480522,2025-11-14 23:58:00,1.763165e+09,0.0,672.865,672.93,672.8500,672.852,288.0,1.0


In [501]:
## CHECK HOW MANY CONSECUTIVE MISS IS HERE (TODO TOMORROW)
new_df_null_data = new_df[new_df['close'].isnull()]

In [502]:
new_df_null_data['diff'] = new_df_null_data['datetime'].diff().dt.total_seconds() / 60

/var/folders/s_/58nvglhd2pl1yr8404nm038w0000gn/T/ipykernel_10442/1567668610.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_null_data['diff'] = new_df_null_data['datetime'].diff().dt.total_seconds() / 60


In [503]:
new_df_null_data['datetime'].apply(lambda x: x.minute)

2          32
6          36
8          38
34          4
43         13
           ..
1443131    47
1443133    49
1446682    58
1448337    33
1477940    56
Name: datetime, Length: 150012, dtype: int64

In [505]:
five_min_interval = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55]
new_df_null_date_5_min = new_df_null_data[new_df_null_data['datetime'].apply(lambda x: True if x.minute in five_min_interval else False)]

In [520]:
new_df_5_min = new_df[new_df['datetime'].apply(lambda x: True if x.minute in five_min_interval else False)]

In [522]:
new_df_5_min[new_df_5_min['close'].isnull()]

,datetime,timestamp,gmtoffset,open,high,low,close,volume,diff
50,2019-01-02 10:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,2019-01-02 10:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,2019-01-02 11:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,2019-01-02 11:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,2019-01-02 11:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1425739,2025-08-19 09:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1430944,2025-08-27 09:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1431804,2025-08-28 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1435299,2025-09-04 09:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [510]:
new_df_null_date_5_min['diff'] = new_df_null_date_5_min['datetime'].diff().dt.total_seconds() / 60

/var/folders/s_/58nvglhd2pl1yr8404nm038w0000gn/T/ipykernel_10442/1205340415.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_null_date_5_min['diff'] = new_df_null_date_5_min['datetime'].diff().dt.total_seconds() / 60


In [543]:
new_df_null_date_5_min['datetime'].dt.year.value_counts()

datetime
2019    6294
2024    6116
2023    5204
2021    4662
2020    3143
2022    3102
2025     293
Name: count, dtype: int64

In [596]:
new_df_null_date_5_min.head(20)

,datetime,timestamp,gmtoffset,open,high,low,close,volume,diff
50,2019-01-02 10:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,2019-01-02 10:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0
95,2019-01-02 11:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0
115,2019-01-02 11:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0
125,2019-01-02 11:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
880,2019-01-03 09:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1325.0
920,2019-01-03 10:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0
925,2019-01-03 10:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
940,2019-01-03 10:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0
950,2019-01-03 10:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0


In [540]:
new_df_null_date_5_min[new_df_null_date_5_min['diff'] == 5]

,datetime,timestamp,gmtoffset,open,high,low,close,volume,diff
925,2019-01-03 10:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
955,2019-01-03 10:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
1805,2019-01-04 10:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
2505,2019-01-04 22:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
2510,2019-01-04 22:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
...,...,...,...,...,...,...,...,...,...
1318799,2025-02-20 22:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
1318804,2025-02-20 22:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
1320544,2025-02-24 22:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
1324074,2025-02-28 23:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0


In [527]:
t = new_df_null_date_5_min[(new_df_null_date_5_min['diff'] == 5) ]

In [ ]:
t['cons_diff'] = t['datetime'].diff().dt.total_seconds() / 60

In [441]:
new_df['diff'] = new_df['datetime'].diff().dt.total_seconds() / 60

In [442]:
null_data_datetime = new_df[new_df['close'].isnull()]['datetime']

In [444]:
new_df[(new_df['close'].isnull()) & (new_df['diff'] > 100)]['datetime'].dt.time.value_counts()

datetime
09:00:00    484
Name: count, dtype: int64

In [453]:
from datetime import time
new_df['datetime'].dt.time == time(9, 0)

0           True
1          False
2          False
3          False
4          False
           ...  
1532389    False
1532390    False
1532391    False
1532392    False
1532393    False
Name: datetime, Length: 1532394, dtype: bool

In [457]:
from datetime import time
new_df[(new_df['close'].isnull()) & (new_df['datetime'].dt.time == time(9, 30))]#['datetime'].dt.time.value_counts()

,datetime,timestamp,gmtoffset,open,high,low,close,volume,diff
2670,2019-01-07 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8909,2019-01-16 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9809,2019-01-17 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
13349,2019-01-24 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
15089,2019-01-28 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
1300692,2024-11-05 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1303391,2024-11-08 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1307886,2024-11-15 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1313284,2024-11-25 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [362]:
final_df.head(20)

,timestamp,gmtoffset,datetime,open,high,low,close,volume,diff
0,1546419600,0,2019-01-02 09:00:00,248.00,248.00,245.03,245.03,10616,NaN
1,1546419660,0,2019-01-02 09:01:00,245.01,245.14,245.00,245.14,8122,1.0
2,1546419720,0,2019-01-02 09:02:00,245.13,245.21,244.96,245.11,2405,1.0
3,1546419780,0,2019-01-02 09:03:00,245.24,245.24,245.23,245.23,277,1.0
4,1546419840,0,2019-01-02 09:04:00,245.30,245.42,245.26,245.41,7650,1.0
5,1546419900,0,2019-01-02 09:05:00,245.41,245.41,245.30,245.35,14651,1.0
6,1546419960,0,2019-01-02 09:06:00,245.24,245.27,245.24,245.25,1489,1.0
7,1546420020,0,2019-01-02 09:07:00,245.34,245.34,245.34,245.34,500,1.0
8,1546420080,0,2019-01-02 09:08:00,245.29,245.34,245.25,245.27,3121,1.0
9,1546420140,0,2019-01-02 09:09:00,245.29,245.46,245.29,245.46,315,1.0


In [200]:
final_df['datetime'] = pd.to_datetime(final_df['datetime'])
df = final_df.sort_values('datetime').set_index('datetime')

bars_5m = df.resample('5T').agg({
    'open': 'first',
    'high': 'max',
    'low':  'min',
    'close':'last',
    'volume':'sum'
}).dropna(subset=['open','close'])  # drop windows with no data at all

/var/folders/s_/58nvglhd2pl1yr8404nm038w0000gn/T/ipykernel_10442/250244206.py:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  bars_5m = df.resample('5T').agg({


In [231]:
bars_5m

,open,high,low,close,volume
2019-01-02 09:00:00,248.0000,248.0000,244.96,245.4100,29070.0
2019-01-02 09:05:00,245.4100,245.4600,245.24,245.4600,20076.0
2019-01-02 09:10:00,245.4600,245.5700,245.44,245.5200,26843.0
2019-01-02 09:15:00,245.5400,245.6500,245.51,245.6400,3106.0
2019-01-02 09:20:00,245.6100,245.8900,245.61,245.7500,4584.0
...,...,...,...,...,...
2025-11-15 00:40:00,673.0799,673.2200,673.03,673.1600,7785.0
2025-11-15 00:45:00,673.1200,673.1517,673.07,673.1517,6381.0
2025-11-15 00:50:00,673.1500,673.2200,673.14,673.2195,7754.0
2025-11-15 00:55:00,673.2100,673.4300,673.20,673.3400,9212.0


In [211]:
np_diff = np.array(bars_5m.index.diff())

In [214]:
bars_5m.index

DatetimeIndex(['2019-01-02 09:00:00', '2019-01-02 09:05:00',
               '2019-01-02 09:10:00', '2019-01-02 09:15:00',
               '2019-01-02 09:20:00', '2019-01-02 09:25:00',
               '2019-01-02 09:30:00', '2019-01-02 09:35:00',
               '2019-01-02 09:40:00', '2019-01-02 09:45:00',
               ...
               '2025-11-15 00:15:00', '2025-11-15 00:20:00',
               '2025-11-15 00:25:00', '2025-11-15 00:30:00',
               '2025-11-15 00:35:00', '2025-11-15 00:40:00',
               '2025-11-15 00:45:00', '2025-11-15 00:50:00',
               '2025-11-15 00:55:00', '2025-11-15 01:00:00'],
              dtype='datetime64[ns]', name='datetime', length=321564, freq=None)

In [221]:
diff_series = bars_5m.index.diff().to_series().dt.total_seconds()/60

In [236]:
diff_series.value_counts()

datetime
5.0       315923
10.0        3236
485.0       1200
15.0         501
480.0        139
3485.0       132
20.0         120
3365.0        77
3480.0        44
25.0          28
3360.0        27
4805.0        19
4925.0        14
490.0         12
30.0          11
1925.0        10
4920.0         9
3545.0         9
3490.0         7
35.0           6
2105.0         5
3425.0         5
40.0           5
45.0           4
4800.0         3
3420.0         3
495.0          2
90.0           1
460.0          1
4985.0         1
4930.0         1
3540.0         1
3495.0         1
3415.0         1
3305.0         1
3475.0         1
3400.0         1
1920.0         1
4980.0         1
Name: count, dtype: int64

In [225]:
# Assume 'bars_5m' is already resampled like before

# 1. Make a full 5-minute index
full_index = pd.date_range(start=bars_5m.index.min(), end=bars_5m.index.max(), freq='5T')

# 2. Reindex to include all 5-minute intervals
bars_5m = bars_5m.reindex(full_index)

# 3. Identify gaps in data
missing_mask = bars_5m['open'].isna()
missing_blocks = missing_mask.astype(int).groupby(missing_mask.ne(missing_mask.shift()).cumsum()).sum()
# this gives you the length of consecutive NaN blocks

/var/folders/s_/58nvglhd2pl1yr8404nm038w0000gn/T/ipykernel_10442/3830645397.py:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_index = pd.date_range(start=bars_5m.index.min(), end=bars_5m.index.max(), freq='5T')


In [228]:
missing_blocks

open
1         0
2        96
3         0
4        96
5         0
         ..
11277     0
11278    95
11279     0
11280    95
11281     0
Name: open, Length: 11281, dtype: int64

## Massive

In [557]:
!pip install massive

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 1.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.1/173.1 kB 3.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [597]:
client = RESTClient("2bp5aheUJHWCzVGSn48CfagsjOyFlfV9")

aggs = []
for a in client.list_aggs(
    "SPY",
    5,
    "minute",
    "2019-01-02",
    "2025-11-12",
    sort="asc",
    limit=120,
):
    aggs.append(a)

print(aggs)

MaxRetryError: HTTPSConnectionPool(host='api.massive.com', port=443): Max retries exceeded with url: /v2/aggs/ticker/SPY/range/5/minute/1700513400000/2025-11-12?cursor=bGltaXQ9MTIwJnNvcnQ9YXNj (Caused by ResponseError('too many 429 error responses'))

In [598]:
datetime.fromtimestamp(aggs[0].timestamp / 1000)

datetime.datetime(2023, 11, 20, 3, 0)

## Databento

In [ ]:
databento = pd.read_csv("xnas-itch-20190101-20251111.ohlcv-1m.csv")

In [634]:
databento['ts_event'] = pd.to_datetime(databento['ts_event'].apply(lambda x: x[:19].replace('T',' ')))

In [639]:
databento['ts_event']

0         2019-01-02 09:00:00
1         2019-01-02 09:01:00
2         2019-01-02 09:02:00
3         2019-01-02 09:03:00
4         2019-01-02 09:04:00
                  ...        
1284851   2025-11-11 23:35:00
1284852   2025-11-11 23:43:00
1284853   2025-11-11 23:44:00
1284854   2025-11-11 23:53:00
1284855   2025-11-11 23:57:00
Name: ts_event, Length: 1284856, dtype: datetime64[ns]

In [646]:
combined = pd.merge(new_df_null_date_5_min[['datetime']], databento, how='left', left_on='datetime', right_on='ts_event')

In [651]:
new_df_5_min

,datetime,timestamp,gmtoffset,open,high,low,close,volume,diff
0,2019-01-02 09:30:00,1.546421e+09,0.0,245.91,245.91,245.880,245.88,979.0,1.0
5,2019-01-02 09:35:00,1.546422e+09,0.0,246.05,246.05,246.000,246.00,468.0,1.0
10,2019-01-02 09:40:00,1.546422e+09,0.0,245.87,245.87,245.630,245.63,1922.0,1.0
15,2019-01-02 09:45:00,1.546422e+09,0.0,245.66,245.66,245.660,245.66,257.0,1.0
20,2019-01-02 09:50:00,1.546423e+09,0.0,245.92,245.93,245.920,245.93,500.0,1.0
...,...,...,...,...,...,...,...,...,...
1480499,2025-11-14 23:35:00,1.763163e+09,0.0,672.98,672.98,672.840,672.86,62.0,1.0
1480504,2025-11-14 23:40:00,1.763164e+09,0.0,672.93,672.93,672.845,672.85,925.0,1.0
1480509,2025-11-14 23:45:00,1.763164e+09,0.0,672.97,672.97,672.850,672.85,89.0,1.0
1480514,2025-11-14 23:50:00,1.763164e+09,0.0,672.90,672.95,672.850,672.85,42.0,1.0


In [617]:
databento['diff'] = databento['ts_event'].diff().dt.total_seconds()/60

In [650]:
new_df_null_date_5_min

,datetime,timestamp,gmtoffset,open,high,low,close,volume,diff
50,2019-01-02 10:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,2019-01-02 10:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0
95,2019-01-02 11:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0
115,2019-01-02 11:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0
125,2019-01-02 11:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
...,...,...,...,...,...,...,...,...,...
1425739,2025-08-19 09:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5770.0
1430944,2025-08-27 09:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11505.0
1431804,2025-08-28 09:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1430.0
1435299,2025-09-04 09:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10095.0


In [649]:
combined[combined['close'].isnull()]

,datetime,ts_event,rtype,publisher_id,instrument_id,open,high,low,close,volume,symbol
0,2019-01-02 10:20:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-02 11:05:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-02 11:25:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-02 11:35:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2019-01-03 09:40:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
28809,2025-08-19 09:55:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28810,2025-08-27 09:40:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28811,2025-08-28 09:30:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28812,2025-09-04 09:45:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
start_time = time(9, 30)
end_time = time(16, 0)
#databento.set_index('ts_event', inplace=True)
databento_databento.between_time(start_time, end_time)

,rtype,publisher_id,instrument_id,open,high,low,close,volume,symbol,diff
ts_event,,,,,,,,,,
2019-01-02 09:30:00,33,2,7294,245.91,245.91,245.880,245.88,800,SPY,1.0
2019-01-02 09:36:00,33,2,7294,246.05,246.05,246.050,246.05,15,SPY,6.0
2019-01-02 09:37:00,33,2,7294,246.04,246.04,246.040,246.04,300,SPY,1.0
2019-01-02 09:39:00,33,2,7294,245.95,245.95,245.950,245.95,169,SPY,2.0
2019-01-02 09:40:00,33,2,7294,245.82,245.82,245.640,245.64,669,SPY,1.0
...,...,...,...,...,...,...,...,...,...,...
2025-11-11 15:56:00,33,2,15144,678.82,678.94,678.730,678.90,16759,SPY,1.0
2025-11-11 15:57:00,33,2,15144,678.89,679.07,678.745,679.07,22462,SPY,1.0
2025-11-11 15:58:00,33,2,15144,679.04,679.16,678.970,679.14,20625,SPY,1.0


In [627]:
databento

,rtype,publisher_id,instrument_id,open,high,low,close,volume,symbol,diff
ts_event,,,,,,,,,,
2019-01-02 09:00:00,33,2,7294,245.38,245.43,245.08,245.08,1655,SPY,NaN
2019-01-02 09:01:00,33,2,7294,245.01,245.19,245.01,245.19,7055,SPY,1.0
2019-01-02 09:02:00,33,2,7294,245.19,245.24,244.96,245.24,964,SPY,1.0
2019-01-02 09:03:00,33,2,7294,245.22,245.22,245.22,245.22,8,SPY,1.0
2019-01-02 09:04:00,33,2,7294,245.26,245.40,245.26,245.40,389,SPY,1.0
...,...,...,...,...,...,...,...,...,...,...
2025-11-11 23:35:00,33,2,15144,682.94,682.95,682.94,682.95,300,SPY,2.0
2025-11-11 23:43:00,33,2,15144,682.97,682.97,682.97,682.97,3,SPY,8.0
2025-11-11 23:44:00,33,2,15144,683.06,683.06,683.06,683.06,3,SPY,1.0


## Kaggle

In [652]:
kaggle_data = pd.read_csv("1_min_SPY_2008-2021.csv")

In [654]:
kaggle_data['date'] = pd.to_datetime(kaggle_data['date'])

In [688]:
kaggle_data['date'].dt.minute

0          30
1          31
2          32
3          33
4          34
           ..
2070829    55
2070830    56
2070831    57
2070832    58
2070833    59
Name: date, Length: 2070834, dtype: int32

In [689]:
kaggle_data_5_min = kaggle_data[kaggle_data['date'].apply(lambda x: True if x.minute in five_min_interval else False)]

In [729]:
kaggle_data_5_min = kaggle_data_5_min.drop_duplicates(subset=['date'])

In [746]:
combined_null = combined[combined['close'].isnull()]

In [747]:
combined_v2 = pd.merge(combined_null[['datetime']], kaggle_data_5_min, how='left', left_on='datetime', right_on='date')

In [760]:
new_df_5_min#['close']

,datetime,timestamp,gmtoffset,open,high,low,close,volume,diff
0,2019-01-02 09:30:00,1.546421e+09,0.0,245.91,245.91,245.880,245.88,979.0,1.0
5,2019-01-02 09:35:00,1.546422e+09,0.0,246.05,246.05,246.000,246.00,468.0,1.0
10,2019-01-02 09:40:00,1.546422e+09,0.0,245.87,245.87,245.630,245.63,1922.0,1.0
15,2019-01-02 09:45:00,1.546422e+09,0.0,245.66,245.66,245.660,245.66,257.0,1.0
20,2019-01-02 09:50:00,1.546423e+09,0.0,245.92,245.93,245.920,245.93,500.0,1.0
...,...,...,...,...,...,...,...,...,...
1480499,2025-11-14 23:35:00,1.763163e+09,0.0,672.98,672.98,672.840,672.86,62.0,1.0
1480504,2025-11-14 23:40:00,1.763164e+09,0.0,672.93,672.93,672.845,672.85,925.0,1.0
1480509,2025-11-14 23:45:00,1.763164e+09,0.0,672.97,672.97,672.850,672.85,89.0,1.0
1480514,2025-11-14 23:50:00,1.763164e+09,0.0,672.90,672.95,672.850,672.85,42.0,1.0


In [761]:
(21335-5507) / 296211

0.05343488256681892

In [755]:
combined_v2#['close'].notnull().sum()

,datetime,Unnamed: 0,date,open,high,low,close,volume,barCount,average
0,2019-01-02 10:20:00,1976594.0,2019-01-02 10:20:00,248.99,249.01,248.91,248.96,1902.0,876.0,248.960
1,2019-01-02 11:05:00,1976639.0,2019-01-02 11:05:00,250.39,250.87,250.38,250.87,4512.0,1999.0,250.586
2,2019-01-02 11:25:00,1976659.0,2019-01-02 11:25:00,250.82,250.82,250.60,250.60,1088.0,529.0,250.677
3,2019-01-02 11:35:00,1976669.0,2019-01-02 11:35:00,250.98,250.99,250.63,250.70,1501.0,929.0,250.758
4,2019-01-03 09:40:00,2031859.0,2019-01-03 09:40:00,247.20,247.53,247.20,247.29,5094.0,2411.0,247.397
...,...,...,...,...,...,...,...,...,...,...
21330,2025-08-19 09:55:00,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21331,2025-08-27 09:40:00,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21332,2025-08-28 09:30:00,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21333,2025-09-04 09:45:00,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
